In [ ]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt

In [ ]:
# load dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (train_images.shape, train_labels.shape))
print('Test: X=%s, y=%s' % (test_images.shape, test_labels.shape))

In [ ]:
# plot first few images
for i in range(1):
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(train_images[i], cmap=plt.get_cmap('gray'))
    print(train_labels[i])
# show the figure
plt.show()

In [ ]:
train_images[0].shape

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Angel Test Model

In [ ]:
angel_model = models.Sequential()
angel_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
angel_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
angel_model.add(layers.AveragePooling2D((2, 2)))
angel_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
angel_model.add(layers.Flatten())
angel_model.add(layers.Dense(64, activation='relu'))
angel_model.add(layers.Dense(10))

In [ ]:
angel_model.summary()

In [ ]:
angel_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

angel_history = angel_model.fit(train_images, train_labels, batch_size=32, epochs=15,
                    callbacks=callback, validation_data=(test_images, test_labels))

#### Evaluate Angel's Model

In [ ]:
plt.plot(angel_history.history['accuracy'], label='accuracy')
plt.plot(angel_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.96, 1])
plt.legend(loc='lower right')

test_loss, test_acc = angel_model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
# Save model
angel_model.save('angel_model')

### John Test Model

In [ ]:
john_model = models.Sequential()
john_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
john_model.add(layers.AveragePooling2D((2, 2)))
john_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
john_model.add(layers.AveragePooling2D((2, 2)))
john_model.add(layers.Flatten())
john_model.add(layers.Dense(64, activation='relu'))
john_model.add(layers.Dense(10))

In [ ]:
john_model.summary()

In [ ]:
john_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

john_history = john_model.fit(train_images, train_labels, batch_size=32, epochs=15,
                    callbacks=callback, validation_data=(test_images, test_labels))

#### Evaluate John's Model

In [ ]:
plt.plot(john_history.history['accuracy'], label='accuracy')
plt.plot(john_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.96, 1])
plt.legend(loc='lower right')

test_loss, test_acc = john_model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
john_model.save('john_model')

### Justin Test Model

In [ ]:
justin_model = models.Sequential()
justin_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
justin_model.add(layers.Dropout(0.2))
justin_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
justin_model.add(layers.AveragePooling2D((2, 2)))
justin_model.add(layers.Flatten())
justin_model.add(layers.Dense(64, activation='relu'))
justin_model.add(layers.Dense(10))

In [ ]:
justin_model.summary()

In [ ]:
justin_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

justin_history = justin_model.fit(train_images, train_labels, batch_size=32, epochs=15,
                    callbacks=callback, validation_data=(test_images, test_labels))

In [ ]:
plt.plot(justin_history.history['accuracy'], label='accuracy')
plt.plot(justin_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.96, 1])
plt.legend(loc='lower right')

test_loss, test_acc = justin_model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
justin_model.save('justin_model')

### Load Models

Here we load models into single ensemble model as list of tuples

In [ ]:
import random
import os

In [ ]:
model_names = ['angel_model', 'john_model', 'justin_model']
trained_models = {}

for name in model_names:
    if os.path.isdir(name):
        trained_models[name] = models.load_model(name)
    else:
        print(f"Invalid model name {name}")


In [ ]:
runs = 5
for _ in range(runs):

    random_index = random.randint(0, len(test_images) -1)
    random_image = test_images[random_index]
    image_label = test_labels[random_index]
    model_predictions = [None] * len(trained_models.keys())
    ensemble_predictions = {}

    # Use each individual model to predict
    for i, (name, model) in enumerate(trained_models.items()):
        model_predictions[i] = np.argmax(model.predict(tf.reshape(random_image, shape=[1, 28, 28, 1])))

    # Vote on final output
    for pred in model_predictions:
        if pred in ensemble_predictions.keys():
            ensemble_predictions[pred] += 1
        else:
            ensemble_predictions[pred] = 1
    
    print(f'True label:\t\t{image_label}')

    # All models agree
    if len(ensemble_predictions) == 1:
        print(f'Ensemble pred. label:\t{list(ensemble_predictions.keys())[0]}')
    
    # 1 model disagrees
    elif len(ensemble_predictions) == 2:
        print(f'Ensemble pred. label:\t{max(ensemble_predictions, key=ensemble_predictions.get)}')

    # all models disagree
    # need to return model with best track record
    else:
        random_pick = random.choice(list(ensemble_predictions.keys()))
        print(f'Ensemble pred. label:\t{random_pick}')


## Creating KKanji Datasets (Midterm)

#### Load datasets into training and validation sets

In [ ]:
new_kkanji_midterm_dataset_train = tf.keras.utils.image_dataset_from_directory(
                                        './midterm_dataset/',
                                        validation_split=0.3,
                                        subset="training",
                                        image_size=(64, 64),
                                        batch_size=32)

In [ ]:
new_kkanji_midterm_dataset_val = tf.keras.utils.image_dataset_from_directory(
                                        './midterm_dataset/',
                                        validation_split=0.3,
                                        subset="validation",
                                        image_size=(64, 64),
                                        batch_size=32)

In [ ]:
import matplotlib.pyplot as plt

class_names = new_kkanji_midterm_dataset_train.class_names

plt.figure(figsize=(10, 10))
for images, labels in new_kkanji_midterm_dataset_train.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in new_kkanji_midterm_dataset_train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

#### Data rescaling 

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_ds = new_kkanji_midterm_dataset_train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

#### Training Performance Optimizations

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = new_kkanji_midterm_dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = new_kkanji_midterm_dataset_train.cache().prefetch(buffer_size=AUTOTUNE)

#### NOTE the change in input/output dimensions from the MNIST dataset

In [ ]:
angel_new_model = models.Sequential()
angel_new_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
angel_new_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
angel_new_model.add(layers.AveragePooling2D((2, 2)))
angel_new_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
angel_new_model.add(layers.Flatten())
angel_new_model.add(layers.Dense(64, activation='relu'))
angel_new_model.add(layers.Dense(50))

In [ ]:
angel_new_model.summary()

In [ ]:
angel_new_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

angel_history = angel_new_model.fit(new_kkanji_midterm_dataset_train, epochs=15,
                    callbacks=callback, validation_data=new_kkanji_midterm_dataset_val)

### NOTE let's all try to generate these plots for our presentation next week pls

In [ ]:
plt.plot(angel_history.history['accuracy'], label='accuracy')
plt.plot(angel_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.75, 1])
plt.legend(loc='lower right')

test_loss, test_acc = angel_new_model.evaluate(new_kkanji_midterm_dataset_val, verbose=2)

In [ ]:
angel_new_model.save('ange_kanji_model')